In [44]:
!pip install pyhf[xmlio,tensorflow,jax,torch]

In [9]:
import tensorflow as tf
tf.config.get_visible_devices()

# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [6]:
tf.config.set_visible_devices([], 'CPU')

RuntimeError: ignored

In [0]:
import numpy as np
import pyhf
import timeit

In [0]:
def generate_source_poisson(n_bins):
    """
    Create the source structure for the given number of bins.
    Sample from a Poisson distribution
    Args:
        n_bins: `list` of number of bins
    Returns:
        source
    """
    np.random.seed(0)  # Fix seed for reproducibility
    binning = [n_bins, -0.5, n_bins + 0.5]
    data = np.random.poisson(120.0, n_bins).tolist()
    bkg = np.random.poisson(100.0, n_bins).tolist()
    bkgerr = np.random.poisson(10.0, n_bins).tolist()
    sig = np.random.poisson(30.0, n_bins).tolist()

    source = {
        "binning": binning,
        "bindata": {"data": data, "bkg": bkg, "bkgerr": bkgerr, "sig": sig},
    }
    return source


def timeit_setup(n_bins, backend_name, mode):
    source = generate_source_poisson(n_bins)
    model = pyhf.simplemodels.hepdata_like(
        source["bindata"]["sig"], source["bindata"]["bkg"], source["bindata"]["bkgerr"]
    )
    pyhf.set_backend(backend_name)
    data = pyhf.tensorlib.astensor(source["bindata"]["data"] + model.config.auxdata)
    test_poi = 1.0
    return test_poi, data, model


def timeit_run(test_mu, data, model):
    # tensorlib is set in timeit_setup
    return pyhf.infer.hypotest(test_mu, data, model)


def generate_setup_string(n_bins, backend_name, mode):
    setup_string = (
        "from __main__ import generate_source_poisson, timeit_setup, timeit_run; "
    )
    setup_string += "import numpy as np; "
    if mode.lower() == "cpu":
        if backend_name.lower() == "tensorflow":
            # Hide the GPU from TensorFlow
            setup_string += "import tensorflow as tf; "
            setup_string += "tf.config.set_visible_devices([], 'GPU'); "
        elif backend_name.lower() == "jax":
          setup_string += "import os; "
          setup_string += "os.environ['JAX_PLATFORM_NAME'] = 'cpu'; "
    elif mode.lower() != "gpu":
        return "ERROR"
    setup_string += (
        f"test_mu, data, model = timeit_setup({n_bins}, '{backend_name}', '{mode}')"
    )
    return setup_string

In [45]:
n_bins = 10
# setup_string = generate_setup_string(n_bins, "numpy", "CPU")
# setup_string = generate_setup_string(n_bins, "tensorflow", "GPU")
# setup_string = generate_setup_string(n_bins, "jax", "GPU")
# setup_string = generate_setup_string(n_bins, "jax", "CPU")
setup_string = generate_setup_string(n_bins, "pytorch", "GPU")
n_runs = 10
exec_time = timeit.timeit('timeit_run(test_mu, data, model)', number=n_runs, setup=setup_string)
exec_time, exec_time/n_runs

(4.1220403590004935, 0.41220403590004934)

In [32]:
np.linspace(200, 400, 3)

array([200., 300., 400.])

In [46]:
results = [("n_bins", "total_time", "n_runs", "time_run")]
# np.linspace(500,7000,14)
bin_range = np.linspace(10, 100, 10)
print(bin_range)

# backend_name = "jax"
backend_name = "pytorch"
mode = "GPU"
# mode = "CPU"
n_runs = 10

for n_bins in bin_range:
    n_bins = int(n_bins)
    setup_string = generate_setup_string(n_bins, backend_name, mode)
    exec_time = timeit.timeit(
        "timeit_run(test_mu, data, model)", number=n_runs, setup=setup_string
    )

    results.append((n_bins, exec_time, n_runs, exec_time / n_runs))
    print(
        "N: {} time: {} time per run: {}".format(n_bins, exec_time, exec_time / n_runs)
    )

bin_range = np.linspace(200, 400, 3)
for n_bins in bin_range:
    n_bins = int(n_bins)
    setup_string = generate_setup_string(n_bins, backend_name, mode)
    exec_time = timeit.timeit(
        "timeit_run(test_mu, data, model)", number=n_runs, setup=setup_string
    )

    results.append((n_bins, exec_time, n_runs, exec_time / n_runs))
    print(
        "N: {} time: {} time per run: {}".format(n_bins, exec_time, exec_time / n_runs)
    )

[ 10.  20.  30.  40.  50.  60.  70.  80.  90. 100.]
N: 10 time: 3.9313855069995043 time per run: 0.3931385506999504
N: 20 time: 7.615285956000662 time per run: 0.7615285956000661
N: 30 time: 11.146268945999509 time per run: 1.114626894599951
N: 40 time: 13.120708712000123 time per run: 1.3120708712000124
N: 50 time: 14.250283641999886 time per run: 1.4250283641999886
N: 60 time: 15.666430123999817 time per run: 1.5666430123999817
N: 70 time: 17.96576649600047 time per run: 1.796576649600047
N: 80 time: 19.814492148999307 time per run: 1.9814492148999308
N: 90 time: 23.184895922000578 time per run: 2.318489592200058
N: 100 time: 26.30782556000031 time per run: 2.630782556000031
N: 200 time: 85.87422372000037 time per run: 8.587422372000038
N: 300 time: 268.8202191139999 time per run: 26.88202191139999
N: 400 time: 619.1974339330009 time per run: 61.91974339330009


In [19]:
setup_string

"from __main__ import generate_source_poisson, timeit_setup, timeit_run; import numpy as np; test_mu, data, model = timeit_setup(500, 'jax', 'GPU')"

In [47]:
results

[('n_bins', 'total_time', 'n_runs', 'time_run'),
 (10, 3.9313855069995043, 10, 0.3931385506999504),
 (20, 7.615285956000662, 10, 0.7615285956000661),
 (30, 11.146268945999509, 10, 1.114626894599951),
 (40, 13.120708712000123, 10, 1.3120708712000124),
 (50, 14.250283641999886, 10, 1.4250283641999886),
 (60, 15.666430123999817, 10, 1.5666430123999817),
 (70, 17.96576649600047, 10, 1.796576649600047),
 (80, 19.814492148999307, 10, 1.9814492148999308),
 (90, 23.184895922000578, 10, 2.318489592200058),
 (100, 26.30782556000031, 10, 2.630782556000031),
 (200, 85.87422372000037, 10, 8.587422372000038),
 (300, 268.8202191139999, 10, 26.88202191139999),
 (400, 619.1974339330009, 10, 61.91974339330009)]

In [0]:
results_numpy_collab = [
    ("n_bins", "total_time", "n_runs", "time_run"),
    (10, 2.2352187829999934, 10, 0.22352187829999934),
    (20, 6.113863463999962, 10, 0.6113863463999962),
    (30, 11.584778862999997, 10, 1.1584778862999996),
    (40, 19.104935302, 10, 1.9104935302000001),
    (50, 26.417424089999997, 10, 2.641742409),
    (60, 36.24537549899998, 10, 3.6245375498999977),
    (70, 45.28580712600001, 10, 4.528580712600001),
    (80, 61.6147713549999, 10, 6.16147713549999),
    (90, 75.02070651600002, 10, 7.502070651600002),
    (100, 94.42928027699998, 10, 9.442928027699997),
]
results_numpy_collab

[('n_bins', 'total_time', 'n_runs', 'time_run'),
 (10, 2.2352187829999934, 10, 0.22352187829999934),
 (20, 6.113863463999962, 10, 0.6113863463999962),
 (30, 11.584778862999997, 10, 1.1584778862999996),
 (40, 19.104935302, 10, 1.9104935302000001),
 (50, 26.417424089999997, 10, 2.641742409),
 (60, 36.24537549899998, 10, 3.6245375498999977),
 (70, 45.28580712600001, 10, 4.528580712600001),
 (80, 61.6147713549999, 10, 6.16147713549999),
 (90, 75.02070651600002, 10, 7.502070651600002),
 (100, 94.42928027699998, 10, 9.442928027699997)]

In [21]:
results_jax_collab_GPU = [('n_bins', 'total_time', 'n_runs', 'time_run'),
 (10, 5.180383256999903, 10, 0.5180383256999903),
 (20, 8.16930957500017, 10, 0.816930957500017),
 (30, 11.560012628000095, 10, 1.1560012628000096),
 (40, 12.531578961999912, 10, 1.2531578961999912),
 (50, 13.671631753999918, 10, 1.3671631753999918),
 (60, 16.720749974999762, 10, 1.6720749974999762),
 (70, 18.179294258000027, 10, 1.8179294258000027),
 (80, 21.993176809000033, 10, 2.1993176809000032),
 (90, 24.47737637299997, 10, 2.447737637299997),
 (100, 28.45362073100023, 10, 2.8453620731000226),
 (200, 106.07671823100009, 10, 10.60767182310001),
 (300, 323.5418221919999, 10, 32.35418221919999),
 (400, 704.4336696660002, 10, 70.44336696660002)]
results_jax_collab_GPU

[('n_bins', 'total_time', 'n_runs', 'time_run'),
 (10, 5.180383256999903, 10, 0.5180383256999903),
 (20, 8.16930957500017, 10, 0.816930957500017),
 (30, 11.560012628000095, 10, 1.1560012628000096),
 (40, 12.531578961999912, 10, 1.2531578961999912),
 (50, 13.671631753999918, 10, 1.3671631753999918),
 (60, 16.720749974999762, 10, 1.6720749974999762),
 (70, 18.179294258000027, 10, 1.8179294258000027),
 (80, 21.993176809000033, 10, 2.1993176809000032),
 (90, 24.47737637299997, 10, 2.447737637299997),
 (100, 28.45362073100023, 10, 2.8453620731000226),
 (200, 106.07671823100009, 10, 10.60767182310001),
 (300, 323.5418221919999, 10, 32.35418221919999),
 (400, 704.4336696660002, 10, 70.44336696660002)]

In [36]:
results_jax_collab_CPU = [('n_bins', 'total_time', 'n_runs', 'time_run'),
 (10, 5.088359718999527, 10, 0.5088359718999527),
 (20, 8.199575525, 10, 0.8199575525),
 (30, 11.334416241999861, 10, 1.133441624199986),
 (40, 12.188941659000193, 10, 1.2188941659000192),
 (50, 13.471506781999778, 10, 1.3471506781999778),
 (60, 16.563559121999788, 10, 1.6563559121999787),
 (70, 17.81586835300004, 10, 1.7815868353000042),
 (80, 21.780555205999917, 10, 2.1780555205999916),
 (90, 24.288290056999358, 10, 2.428829005699936),
 (100, 27.980713243000537, 10, 2.7980713243000537),
 (200, 103.16081419999955, 10, 10.316081419999955),
 (300, 323.0815654040007, 10, 32.30815654040007),
 (400, 707.8526506119997, 10, 70.78526506119996)]
results_jax_collab_CPU

[('n_bins', 'total_time', 'n_runs', 'time_run'),
 (10, 5.088359718999527, 10, 0.5088359718999527),
 (20, 8.199575525, 10, 0.8199575525),
 (30, 11.334416241999861, 10, 1.133441624199986),
 (40, 12.188941659000193, 10, 1.2188941659000192),
 (50, 13.471506781999778, 10, 1.3471506781999778),
 (60, 16.563559121999788, 10, 1.6563559121999787),
 (70, 17.81586835300004, 10, 1.7815868353000042),
 (80, 21.780555205999917, 10, 2.1780555205999916),
 (90, 24.288290056999358, 10, 2.428829005699936),
 (100, 27.980713243000537, 10, 2.7980713243000537),
 (200, 103.16081419999955, 10, 10.316081419999955),
 (300, 323.0815654040007, 10, 32.30815654040007),
 (400, 707.8526506119997, 10, 70.78526506119996)]

In [48]:
results_pytorch_collab_GPU = [('n_bins', 'total_time', 'n_runs', 'time_run'),
 (10, 3.9313855069995043, 10, 0.3931385506999504),
 (20, 7.615285956000662, 10, 0.7615285956000661),
 (30, 11.146268945999509, 10, 1.114626894599951),
 (40, 13.120708712000123, 10, 1.3120708712000124),
 (50, 14.250283641999886, 10, 1.4250283641999886),
 (60, 15.666430123999817, 10, 1.5666430123999817),
 (70, 17.96576649600047, 10, 1.796576649600047),
 (80, 19.814492148999307, 10, 1.9814492148999308),
 (90, 23.184895922000578, 10, 2.318489592200058),
 (100, 26.30782556000031, 10, 2.630782556000031),
 (200, 85.87422372000037, 10, 8.587422372000038),
 (300, 268.8202191139999, 10, 26.88202191139999),
 (400, 619.1974339330009, 10, 61.91974339330009)]
results_pytorch_collab_GPU

[('n_bins', 'total_time', 'n_runs', 'time_run'),
 (10, 3.9313855069995043, 10, 0.3931385506999504),
 (20, 7.615285956000662, 10, 0.7615285956000661),
 (30, 11.146268945999509, 10, 1.114626894599951),
 (40, 13.120708712000123, 10, 1.3120708712000124),
 (50, 14.250283641999886, 10, 1.4250283641999886),
 (60, 15.666430123999817, 10, 1.5666430123999817),
 (70, 17.96576649600047, 10, 1.796576649600047),
 (80, 19.814492148999307, 10, 1.9814492148999308),
 (90, 23.184895922000578, 10, 2.318489592200058),
 (100, 26.30782556000031, 10, 2.630782556000031),
 (200, 85.87422372000037, 10, 8.587422372000038),
 (300, 268.8202191139999, 10, 26.88202191139999),
 (400, 619.1974339330009, 10, 61.91974339330009)]